In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit.circuit.library import CXGate, U3Gate, Measure
from qiskit.circuit.library import HGate, RZGate, RXGate, RYGate, SGate, TGate, XGate, YGate, ZGate, CZGate, SwapGate, iSwapGate
from qiskit.quantum_info import Statevector, state_fidelity
from qiskit.result import marginal_counts
from qiskit_aer.noise import NoiseModel, thermal_relaxation_error, depolarizing_error, ReadoutError
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime.fake_provider import (
    FakeLimaV2, FakeNairobiV2, FakeJakartaV2, FakeYorktownV2, FakeSantiagoV2,
    FakeManilaV2, FakeBogotaV2, FakeQuitoV2, FakeBelemV2, FakeOslo , FakeLondonV2
)
from qiskit_ibm_runtime.fake_provider import (
    FakeTorontoV2, FakeBrooklynV2, FakeMontrealV2, FakeMumbaiV2, FakeWashingtonV2
)
from qiskit_ibm_runtime.fake_provider import (
    FakeLondonV2, FakeManhattanV2, FakeMarrakesh, FakeMelbourneV2, FakeOsaka,
    FakeOurenseV2, FakeParisV2, FakePeekskill, FakePerth, FakePoughkeepsieV2,
    FakePrague, FakeQuebec, FakeQuitoV2, FakeRochesterV2, FakeRomeV2,
    FakeSantiagoV2, FakeSherbrooke, FakeSingaporeV2, FakeSydneyV2,
    FakeValenciaV2, FakeVigoV2, FakeYorktownV2
)
from scipy.spatial.distance import jensenshannon
import numpy as np
import random
import os

# -------------PART 1------------------
#DEFINING THE SIMULATION PARAMETERS FOR FAKE QUANTUM PROCESSORS AND GENERATING RANDOM QUANTUM CIRCUITS WITH THEIR FEATURES


#Defining each processor simulation parameters
"""each key is the namae of a fake processor(quantum hardware) and the tuple infront of it is
for parameters that together they will simulate the behavior of that processor """
processors = {
    "ibmq_lima": (FakeLimaV2, 70e3, 50e3, 0.025, 0.0018, 0.01, 70e-9, 300e-9, 500e-9),
    "ibmq_belem": (FakeBelemV2, 65e3, 45e3, 0.03, 0.002, 0.012, 80e-9, 310e-9, 500e-9),
    "ibmq_quito": (FakeQuitoV2, 68e3, 48e3, 0.026, 0.0022, 0.011, 75e-9, 305e-9, 500e-9),
    "ibmq_bogota": (FakeBogotaV2, 66e3, 46e3, 0.029, 0.0021, 0.013, 78e-9, 315e-9, 500e-9),
    "ibmq_oslo": (FakeOslo, 72e3, 52e3, 0.022, 0.0017, 0.0095, 65e-9, 290e-9, 500e-9),
    "ibmq_manila": (FakeManilaV2, 74e3, 54e3, 0.021, 0.0015, 0.009, 62e-9, 285e-9, 500e-9),
    "ibmq_santiago": (FakeSantiagoV2, 69e3, 49e3, 0.027, 0.0019, 0.0105, 72e-9, 295e-9, 500e-9),
    "ibmq_yorktown": (FakeYorktownV2, 60e3, 40e3, 0.035, 0.0025, 0.014, 85e-9, 320e-9, 500e-9),
    "ibmq_jakarta": (FakeJakartaV2, 73e3, 53e3, 0.023, 0.0016, 0.0092, 68e-9, 292e-9, 500e-9),
    "ibmq_nairobi": (FakeNairobiV2, 75e3, 55e3, 0.020, 0.0014, 0.0088, 60e-9, 280e-9, 500e-9),
}
processors["ibmq_toronto"] = (FakeTorontoV2, 80e3, 60e3, 0.018, 0.0012, 0.007, 50e-9, 250e-9, 500e-9)
processors["ibmq_brooklyn"] = (FakeBrooklynV2, 85e3, 65e3, 0.017, 0.0011, 0.006, 45e-9, 240e-9, 500e-9)
processors["ibmq_montreal"] = (FakeMontrealV2, 78e3, 58e3, 0.019, 0.0013, 0.0075, 52e-9, 255e-9, 500e-9)
processors["ibmq_mumbai"] = (FakeMumbaiV2, 83e3, 63e3, 0.016, 0.001, 0.0065, 48e-9, 245e-9, 500e-9)
processors["ibmq_washington"] = (FakeWashingtonV2, 90e3, 70e3, 0.015, 0.0009, 0.0055, 40e-9, 230e-9, 500e-9)
processors["ibmq_london"] = (FakeLondonV2,76e3,56e3,0.024,0.0016,0.0098,66e-9,290e-9,500e-9)
processors.update({
    "ibmq_london": (FakeLondonV2, 76e3, 56e3, 0.024, 0.0016, 0.0098, 66e-9, 290e-9, 500e-9),
    "ibmq_manhattan": (FakeManhattanV2, 88e3, 68e3, 0.014, 0.0009, 0.005, 40e-9, 220e-9, 500e-9),
    "ibmq_marrakesh": (FakeMarrakesh, 75e3, 55e3, 0.020, 0.0014, 0.0088, 60e-9, 280e-9, 500e-9),
    "ibmq_melbourne": (FakeMelbourneV2, 70e3, 50e3, 0.025, 0.0018, 0.010, 70e-9, 300e-9, 500e-9),
    "ibmq_osaka": (FakeOsaka, 72e3, 52e3, 0.022, 0.0017, 0.0095, 65e-9, 290e-9, 500e-9),
    "ibmq_ourense": (FakeOurenseV2, 68e3, 48e3, 0.026, 0.0022, 0.011, 75e-9, 305e-9, 500e-9),
    "ibmq_paris": (FakeParisV2, 74e3, 54e3, 0.021, 0.0015, 0.009, 62e-9, 285e-9, 500e-9),
    "ibmq_peekskill": (FakePeekskill, 80e3, 60e3, 0.018, 0.0012, 0.007, 50e-9, 250e-9, 500e-9),
    "ibmq_perth": (FakePerth, 78e3, 58e3, 0.019, 0.0013, 0.0075, 52e-9, 255e-9, 500e-9),
    "ibmq_poughkeepsie": (FakePoughkeepsieV2, 83e3, 63e3, 0.016, 0.001, 0.0065, 48e-9, 245e-9, 500e-9),
    "ibmq_prague": (FakePrague, 85e3, 65e3, 0.017, 0.0011, 0.006, 45e-9, 240e-9, 500e-9),
    "ibmq_quebec": (FakeQuebec, 90e3, 70e3, 0.015, 0.0009, 0.0055, 40e-9, 230e-9, 500e-9),
    "ibmq_quito": (FakeQuitoV2, 68e3, 48e3, 0.026, 0.0022, 0.011, 75e-9, 305e-9, 500e-9),
    "ibmq_rochester": (FakeRochesterV2, 86e3, 66e3, 0.016, 0.001, 0.006, 46e-9, 235e-9, 500e-9),
    "ibmq_rome": (FakeRomeV2, 69e3, 49e3, 0.027, 0.0019, 0.0105, 72e-9, 295e-9, 500e-9),
    "ibmq_santiago": (FakeSantiagoV2, 69e3, 49e3, 0.027, 0.0019, 0.0105, 72e-9, 295e-9, 500e-9),
    "ibmq_sherbrooke": (FakeSherbrooke, 92e3, 72e3, 0.013, 0.0008, 0.005, 38e-9, 225e-9, 500e-9),
    "ibmq_singapore": (FakeSingaporeV2, 91e3, 71e3, 0.014, 0.0009, 0.0052, 39e-9, 228e-9, 500e-9),
    "ibmq_sydney": (FakeSydneyV2, 93e3, 73e3, 0.012, 0.0007, 0.0048, 37e-9, 222e-9, 500e-9),
    "ibmq_valencia": (FakeValenciaV2, 70e3, 50e3, 0.025, 0.0018, 0.010, 70e-9, 300e-9, 500e-9),
    "ibmq_vigo": (FakeVigoV2, 66e3, 46e3, 0.029, 0.0021, 0.013, 78e-9, 315e-9, 500e-9),
    "ibmq_yorktown": (FakeYorktownV2, 60e3, 40e3, 0.035, 0.0025, 0.014, 85e-9, 320e-9, 500e-9)
})

#This function analyzes a given quantum circuit and extracts numerical features from it
def extract_features(circuit):
    ops = circuit.count_ops()
    qubit_count = circuit.num_qubits #qubit number
    depth = circuit.depth() #gate layers
    cx_count = ops.get("cx", 0) #number of CX gate (CNOT)
    u3_count = ops.get("u3", 0) #number of U3 gate
    entangling_gates = cx_count #entangling gates is the CX gate (because it's the most commonly used gate for this purpose and it's for simplicity )
    entanglement_ratio = entangling_gates / depth if depth > 0 else 0 #ratio of entangling gates to the total depth
    x_count = ops.get("x", 0) #number of X gate
    y_count = ops.get("y", 0) #number of Y gate
    z_count = ops.get("z", 0) #number of Z gate
    h_count = ops.get("h", 0) #number of H gate
    rx_count = ops.get("rx", 0) #number of Rx gate
    ry_count = ops.get("ry", 0) #number of Ry gate
    rz_count = ops.get("rz", 0) #number of Rz gate
    s_count = ops.get("s", 0) #number of s gate
    t_count = ops.get("t", 0) #number of t gate
    total_gate_count = sum(ops.values())
    total_1q_gates = (
        ops.get("u1", 0) + ops.get("u2", 0) + u3_count + x_count + y_count + z_count +
        h_count + rx_count + ry_count + rz_count + s_count + t_count
    )
    gate_diversity = len(ops) #the number of distinct gates 
    #these metrics are important to determine the complexity and general structure of quantum circuits
    cx_per_qubit = cx_count / qubit_count if qubit_count > 0 else 0
    width_depth_ratio = qubit_count / depth if depth > 0 else 0
    cx_u3_ratio = cx_count / (u3_count + 1)  #that +1 is for not getting 0 in denominator
    single_qubit_ratio = total_1q_gates / total_gate_count if total_gate_count > 0 else 0
    """Additional note:
    - CX gates per qubit: how many entangling gates(e.g. CNOT) are applied on average for each qubit
    - CX/U3: comparing the entangling gates to 1-qubit gates. higher the CX/U3 ratio ~ higher the entangling gates relative to the number of 1-qubit rotations
    - single-Qubit Gate ratio: 1-qubit gates (e.g. U3, X, Y, Z, H, ...) relative to the total gate numbers
    """
    return [qubit_count, depth, cx_count, u3_count, entanglement_ratio,x_count, y_count, z_count,h_count, rx_count, ry_count, rz_count, s_count, t_count,total_gate_count, gate_diversity, cx_per_qubit,width_depth_ratio, cx_u3_ratio, single_qubit_ratio]
single_qubit_gates = [U3Gate, HGate, RZGate, RXGate, RYGate, SGate, TGate, XGate, YGate, ZGate]
entangling_gates = [CXGate, CZGate, SwapGate ]
#This function creates a random quantum circuit to make different circuits
def generate_random_circuit(qubit_range=(4,11), max_depth=25):
    num_qubits = random.randint(*qubit_range)
    depth = random.randint(5, max_depth)
    qc = QuantumCircuit(num_qubits)
    for _ in range(depth):
        for q in range(num_qubits):
            if random.random() < 0.7: #With 70% chance, it applies a random single-qubit gate (e.g. U3, RX, RY, ...)
                gate = random.choice(single_qubit_gates)
                if gate in [U3Gate, RXGate, RYGate, RZGate]:
                    if gate == U3Gate:
                        qc.append(U3Gate(*np.random.rand(3) * 2 * np.pi), [q])
                    else:
                        qc.append(gate(*np.random.rand(1) * 2 * np.pi), [q])
                else:
                    qc.append(gate(), [q])
        for _ in range(num_qubits // 2):
            q1, q2 = random.sample(range(num_qubits), 2)
            if random.random() < 0.5: #With 50% chance, applies a CNOT (CX) gate between pairs of qubits
                gate = random.choice(entangling_gates)
                qc.append(gate(), [q1, q2])
    qc.measure_all()
    return qc
#This function, build_noise_model, is used to create a noise model for simulating noise effects on quantum circuits.
def build_noise_model(n_qubits, t1, t2, readout_error, gate_error_1q, gate_error_2q,
                      gate_time_1q, gate_time_2q, meas_time, coupling_map, basis_gates):
    noise_model = NoiseModel()
    # Readout error model
    ro = ReadoutError([[1 - readout_error, readout_error], [readout_error, 1 - readout_error]])
    # List of all 1-qubit gates you use in circuits
    noisy_1q_gates = ['u1', 'u2', 'u3', 'h', 'rx', 'ry', 'rz', 's', 't', 'x', 'y', 'z']
    # Apply noise to single-qubit gates and readout
    for q in range(n_qubits):
        relax_1q = thermal_relaxation_error(t1, t2, gate_time_1q) #thermal relaxation error (T1, T2)  for a single qubit
        #depolarizing error for a single qubit (when a qubit loses its coherence and its state becomes random with a certain probability)
        depol_1q = depolarizing_error(gate_error_1q, 1) 
        combined_error_1q = relax_1q.compose(depol_1q)
        #adding (thermal relaxation + depolarization) to the noise model for the specified qubit
        noise_model.add_quantum_error(combined_error_1q, noisy_1q_gates, [q])
        noise_model.add_readout_error(ro, [q])
    #apply noise to two-qubit CX gates based on the coupling map(describes which qubits can interact with each other)
    for q0, q1 in coupling_map:
        relax_2q = thermal_relaxation_error(t1, t2, gate_time_2q).tensor(
            thermal_relaxation_error(t1, t2, gate_time_2q)
        )
        depol_2q = depolarizing_error(gate_error_2q, 2)
        combined_error_2q = relax_2q.compose(depol_2q)
        noise_model.add_quantum_error(combined_error_2q, ['cx'], [q0, q1])

    return noise_model

#Creating noise models for Processors
noise_models = {}
for name, (FakeBackend, *params) in processors.items():
    backend = FakeBackend()
    noise_models[name] = build_noise_model(
        backend.configuration().num_qubits, *params,
        coupling_map=backend.configuration().coupling_map,
        basis_gates=backend.configuration().basis_gates
    )
#simulating on noise free backend & calculating fidelity
#The function calculates the Jensen-Shannon divergence (fidelity) between the probability distributions of ideal and noisy measurement 
def compute_fidelity(ideal_counts, noisy_counts, shots=1024):
    keys = set(ideal_counts.keys()).union(noisy_counts.keys())
    p = np.array([ideal_counts.get(k, 0)/shots for k in keys])
    q = np.array([noisy_counts.get(k, 0)/shots for k in keys])
    return 1 - jensenshannon(p, q)


# -------------PART 2------------------
#SIMULATING RANDOM QUANTUM CIRCUITS ON DIFFERENT QUANTUM BACKENDS, COMPUTING THEIR FIDELITY BY COMPARING NOISY AND IDEAL RESULTS, AND SELECTING THE BEST BACKEND FOR EACH CIRCUIT

#number of shots(measurements) for each simulation. more shots ~ more statistical reliability ~ more computational cost
shots = 256
#number of random quantum circuits to simulate
NUM_CIRCUITS = 800
#the number of times to run the simulation on each backend (quantum simulator)
RUNS_PER_SIMULATOR = 2
final_dict = {}

#simulating random circuits
for i in range(NUM_CIRCUITS):
    print(f"Simulating circuit {i+1}/{NUM_CIRCUITS}") #the progress of the simulation
    circ = generate_random_circuit() #a new random quantum circuit
    features = extract_features(circ)
    feature_key = tuple(features) #tuple is used as a key for storing the best simulator for this circuit in final_dict

    #ideal simulation(noise free) (to avoid unsupported gates)
    transpiled_ideal = transpile(circ) #to ensure the quantum gates are supported on the chosen backen
    #initializes an ideal quantum simulator from Qiskit Aer.it's noisefree & is used to get the ideal results of the quantum circuit
    ideal_sim = AerSimulator()
    ideal_counts = ideal_sim.run(transpiled_ideal, shots=shots).result().get_counts()

#simulating with noise models on different backends
    avg_fidelities = {}
    for name, (FakeBackend, *_params) in processors.items():
        """note: The *_params is for unpacking the values from the tuple without explicitly naming them"""
        backend = FakeBackend()
        #each processor only support a limited qubit
        """if the number of qubits in the generated quantum circuit exceeds the number of qubits available on the current backend, the loop skip that backend"""
        if circ.num_qubits > backend.configuration().num_qubits:
            continue
#simulating on noisy backend & calculating fidelity
        sim = AerSimulator(
            noise_model=noise_models[name],
            basis_gates=backend.configuration().basis_gates,
            coupling_map=backend.configuration().coupling_map
        )

        fidelities = []
        for _ in range(RUNS_PER_SIMULATOR):
            transpiled = transpile(circ, backend=backend)
            result = sim.run(transpiled, shots=shots).result()
            noisy_counts = result.get_counts()
        #Computes the fidelity(with Jensen-Shannon divergence) between the ideal & the noisy from the simulation
            fidelity = compute_fidelity(ideal_counts, noisy_counts, shots)
            fidelities.append(fidelity)
        
        #the average fidelity for this backend
        if fidelities:
            avg_fidelities[name] = np.mean(fidelities)
    #the backend with the highest average fidelity
    if avg_fidelities:
        best_sim = max(avg_fidelities, key=avg_fidelities.get)
        final_dict[feature_key] = best_sim

# -------------PART 3------------------
#FORMATS THE SIMULATION RESULTS INTO A TABLE AND SAVES THEM TO A TEXT FILE


output_file = "simulator_results.txt" #the name of the file

#Headers for the Table
headers = [
    "Circuit #", "Qubits", "Depth", "CX", "U3", "Ent. Ratio",
    "X", "Y", "Z", "H", "RX", "RY", "RZ", "S", "T",
    "Total Gates", "Diversity", "CX/Qubit", "Width/Depth", "CX/U3", "1Q Ratio",
    "Best Simulator"
]

#the width of each column
col_widths = [
    10, 7, 7, 6, 6, 12, 4, 4, 4, 4, 5, 5, 5, 4, 4,
    13, 10, 10, 13, 8, 9, 16
]

#borders of the table
def make_line():
    return "+" + "+".join(["-" * w for w in col_widths]) + "+"

#opening the simulator_results.txt file for writing
with open(output_file, "w") as f: #w stands for the write mode
    f.write(make_line() + "\n") #top border
    f.write("|" + "|".join(h.center(w) for h, w in zip(headers, col_widths)) + "|\n") #the header row
    f.write(make_line() + "\n")

    #data rows
    for i, (features, best_sim) in enumerate(final_dict.items(), 1):
        row_data = [str(i)] + [f"{x:.2f}" if isinstance(x, float) else str(x) for x in features] + [best_sim]
        f.write("|" + "|".join(d.center(w) for d, w in zip(row_data, col_widths)) + "|\n")

    f.write(make_line() + "\n") #bottom border 

#file path where the results have been saved
print(f"Results written to: {os.path.abspath(output_file)}")

Simulating circuit 1/1
Results written to: f:\Microsoft VS Code\simulator_results.txt
